# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Haoyang"
# Set the data and output directories
DATA_ROOT = '/media/techt/DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Vitamin D Levels'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [ ]:
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = None
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)
clinical_data_cols[:10]

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

In [ ]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Vitamin-D-Levels"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE118723', 'GSE123993', 'GSE129604', 'GSE86406']

Repeat the below steps for all the accession numbers

In [4]:
cohort = accession_num = "GSE123993"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/media/techt/DATA/GEO/Vitamin-D-Levels/GSE123993/GSE123993_family.soft.gz',
 '/media/techt/DATA/GEO/Vitamin-D-Levels/GSE123993/GSE123993_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [5]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"No effect of calcifediol supplementation on skeletal muscle transcriptome in vitamin D deficient frail older adults."
!Series_summary	"Vitamin D deficiency is common among older adults and has been linked to muscle weakness. Vitamin D supplementation has been proposed as a strategy to improve muscle function in older adults. The aim of this study was to investigate the effect of calcifediol (25-hydroxycholecalciferol) on whole genome gene expression in skeletal muscle of vitamin D deficient frail older adults. A double-blind placebo controlled trial was conducted in vitamin D deficient frail older adults (aged above 65), characterized by blood 25-hydroxycholecalciferol concentrations between 20 and 50 nmol/L. Subjects were randomized across the placebo group (n=12) and the calcifediol group (n=10, 10 µg per day). Muscle biopsies were obtained before and after six months of calcifediol or placebo supplementation and subjected to whole genome gene expression profiling usin

In [6]:
clinical_data.head()

,!Sample_geo_accession,GSM3518336,GSM3518337,GSM3518338,GSM3518339,GSM3518340,GSM3518341,GSM3518342,GSM3518343,GSM3518344,...,GSM3518370,GSM3518371,GSM3518372,GSM3518373,GSM3518374,GSM3518375,GSM3518376,GSM3518377,GSM3518378,GSM3518379
0,!Sample_characteristics_ch1,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,...,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle
1,!Sample_characteristics_ch1,Sex: Male,Sex: Male,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Male,Sex: Male,Sex: Male,...,Sex: Male,Sex: Male,Sex: Male,Sex: Male,Sex: Male,Sex: Male,Sex: Female,Sex: Female,Sex: Female,Sex: Female
2,!Sample_characteristics_ch1,subject id: 3087,subject id: 3087,subject id: 3088,subject id: 3088,subject id: 3090,subject id: 3090,subject id: 3106,subject id: 3106,subject id: 3178,...,subject id: 3498,subject id: 3498,subject id: 3516,subject id: 3516,subject id: 3614,subject id: 3614,subject id: 3695,subject id: 3695,subject id: 3731,subject id: 3731
3,!Sample_characteristics_ch1,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: Placebo,...,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: Placebo,intervention group: Placebo,intervention group: 25-hydroxycholecalciferol ...,intervention group: 25-hydroxycholecalciferol ...,intervention group: Placebo,intervention group: Placebo
4,!Sample_characteristics_ch1,time of sampling: before intervention (baseline),time of sampling: after intervention,time of sampling: before intervention (baseline),time of sampling: after intervention,time of sampling: before intervention (baseline),time of sampling: after intervention,time of sampling: before intervention (baseline),time of sampling: after intervention,time of sampling: before intervention (baseline),...,time of sampling: before intervention (baseline),time of sampling: after intervention,time of sampling: before intervention (baseline),time of sampling: after intervention,time of sampling: before intervention (baseline),time of sampling: after intervention,time of sampling: before intervention (baseline),time of sampling: after intervention,time of sampling: before intervention (baseline),time of sampling: after intervention


In [7]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: muscle'],
 1: ['Sex: Male', 'Sex: Female'],
 2: ['subject id: 3087',
  'subject id: 3088',
  'subject id: 3090',
  'subject id: 3106',
  'subject id: 3178',
  'subject id: 3241',
  'subject id: 3258',
  'subject id: 3279',
  'subject id: 3283',
  'subject id: 3295',
  'subject id: 3322',
  'subject id: 3341',
  'subject id: 3360',
  'subject id: 3361',
  'subject id: 3375',
  'subject id: 3410',
  'subject id: 3430',
  'subject id: 3498',
  'subject id: 3516',
  'subject id: 3614',
  'subject id: 3695',
  'subject id: 3731'],
 3: ['intervention group: 25-hydroxycholecalciferol (25(OH)D3)',
  'intervention group: Placebo'],
 4: ['time of sampling: before intervention (baseline)',
  'time of sampling: after intervention']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [8]:

f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. Do remember to separate the value by colon ':' first, as the row header should not be included in the returned value. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Vitamin D Levels\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Vitamin D Levels\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The ke

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [9]:
age_row = gender_row = None
convert_age = convert_gender = None

In [10]:
is_gene_availabe = True
trait_row = 3
#age_row = 1
gender_row = 1

trait_type = 'binary'

In [11]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [12]:
# Verify and use the functions generated by GPT
def convert_trait(value):
    """Converts the value related to Vitamin D supplementation to binary.

    Args:
    value (str): The input value in the format 'intervention group: <value>'

    Returns:
    binary: 1 for calcifediol supplementation, 0 for placebo, None for unknown values
    """
    if not value:
        return None

    # Separate the value from the header
    _, intervention = value.split(': ', 1)

    if intervention == '25-hydroxycholecalciferol (25(OH)D3)':
        return 1  # Vitamin D supplementation
    elif intervention == 'Placebo':
        return 0  # No Vitamin D supplementation
    else:
        return None  # Unknown value

def convert_gender(value):
    """Converts the gender value to binary.

    Args:
    value (str): The input value in the format 'Sex: <value>'

    Returns:
    binary: 1 for Male, 0 for Female, None for unknown values
    """
    if not value:
        return None

    # Separate the value from the header
    _, gender = value.split(': ', 1)

    if gender == 'Male':
        return 1  # Male
    elif gender == 'Female':
        return 0  # Female
    else:
        return None  # Unknown value

# Test the functions with example values
test_trait = "intervention group: 25-hydroxycholecalciferol (25(OH)D3)"
test_gender = "Sex: Female"

convert_trait_result = convert_trait(test_trait)
convert_gender_result = convert_gender(test_gender)

convert_trait_result, convert_gender_result




(1, 0)

In [13]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM3518336,GSM3518337,GSM3518338,GSM3518339,GSM3518340,GSM3518341,GSM3518342,GSM3518343,GSM3518344,GSM3518345,...,GSM3518370,GSM3518371,GSM3518372,GSM3518373,GSM3518374,GSM3518375,GSM3518376,GSM3518377,GSM3518378,GSM3518379
Vitamin D Levels,1,1,1,1,1,1,1,1,0,0,...,1,1,1,1,0,0,1,1,0,0
Gender,1,1,0,0,0,0,1,1,1,1,...,1,1,1,1,1,1,0,0,0,0


### Genetic data preprocessing and final filtering

In [14]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM3518336,GSM3518337,GSM3518338,GSM3518339,GSM3518340,GSM3518341,GSM3518342,GSM3518343,GSM3518344,GSM3518345,...,GSM3518370,GSM3518371,GSM3518372,GSM3518373,GSM3518374,GSM3518375,GSM3518376,GSM3518377,GSM3518378,GSM3518379
ID,,,,,,,,,,,,,,,,,,,,,
16650001,1.512274,0.253623,2.664750,1.312637,0.675071,1.010733,2.288005,1.868232,1.509000,1.749677,...,2.000858,0.682592,0.876719,0.763386,0.504208,0.745772,2.164791,2.036223,1.317688,1.750596
16650003,1.003784,1.028232,1.341918,1.114636,1.802068,1.946301,1.718234,2.383265,0.789789,1.545537,...,1.175455,1.221579,0.915122,1.008250,1.256585,1.793582,1.560620,1.797027,1.578021,0.752845
16650005,0.604331,1.397437,1.651186,3.274191,0.394109,1.676047,2.146871,2.962444,1.629054,1.412083,...,1.407318,2.406541,3.134922,1.862797,1.704497,1.060720,3.686494,2.485064,1.110987,3.715562
16650007,1.058137,0.588526,1.149379,0.761508,2.417583,0.721435,1.876002,2.209180,2.336481,0.683767,...,0.456273,0.687521,0.356791,0.863351,0.942087,1.214890,1.553857,1.393150,1.699157,0.787946
16650009,0.469632,0.698155,0.888779,1.154360,0.859562,0.659498,0.611699,0.741379,0.490208,0.466021,...,1.058347,0.643452,0.528055,0.527859,0.938790,0.524942,0.508296,0.932883,0.878911,0.808085


In [15]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['16650001',
 '16650003',
 '16650005',
 '16650007',
 '16650009',
 '16650011',
 '16650013',
 '16650015',
 '16650017',
 '16650019',
 '16650021',
 '16650023',
 '16650025',
 '16650027',
 '16650029',
 '16650031',
 '16650033',
 '16650035',
 '16650037',
 '16650041']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [16]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['16650001', '16650003', '16650005', '16650007', '16650009', '16650011', '16650013', '16650015', '16650017', '16650019', '16650021', '16650023', '16650025', '16650027', '16650029', '16650031', '16650033', '16650035', '16650037', '16650041']\n"


If not required, jump directly to the gene normalization step

In [17]:
requires_gene_mapping = True

In [18]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [19]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    ['16650001', '16650003', '16650005', '16650007', '16650009', '16650011', '16650013', '16650015', '16650017', '16650019', '16650021', '16650023', '16650025', '16650027', '16650029', '16650031', '16650033', '16650035', '16650037', '16650041']
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['

In [20]:
genetic_data

,GSM3518336,GSM3518337,GSM3518338,GSM3518339,GSM3518340,GSM3518341,GSM3518342,GSM3518343,GSM3518344,GSM3518345,...,GSM3518370,GSM3518371,GSM3518372,GSM3518373,GSM3518374,GSM3518375,GSM3518376,GSM3518377,GSM3518378,GSM3518379
ID,,,,,,,,,,,,,,,,,,,,,
16650001,1.512274,0.253623,2.664750,1.312637,0.675071,1.010733,2.288005,1.868232,1.509000,1.749677,...,2.000858,0.682592,0.876719,0.763386,0.504208,0.745772,2.164791,2.036223,1.317688,1.750596
16650003,1.003784,1.028232,1.341918,1.114636,1.802068,1.946301,1.718234,2.383265,0.789789,1.545537,...,1.175455,1.221579,0.915122,1.008250,1.256585,1.793582,1.560620,1.797027,1.578021,0.752845
16650005,0.604331,1.397437,1.651186,3.274191,0.394109,1.676047,2.146871,2.962444,1.629054,1.412083,...,1.407318,2.406541,3.134922,1.862797,1.704497,1.060720,3.686494,2.485064,1.110987,3.715562
16650007,1.058137,0.588526,1.149379,0.761508,2.417583,0.721435,1.876002,2.209180,2.336481,0.683767,...,0.456273,0.687521,0.356791,0.863351,0.942087,1.214890,1.553857,1.393150,1.699157,0.787946
16650009,0.469632,0.698155,0.888779,1.154360,0.859562,0.659498,0.611699,0.741379,0.490208,0.466021,...,1.058347,0.643452,0.528055,0.527859,0.938790,0.524942,0.508296,0.932883,0.878911,0.808085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17127713,4.414351,3.902043,4.123132,4.505749,5.171712,5.388863,4.414378,4.460149,4.871354,4.244545,...,4.820871,4.876049,4.384864,4.520500,4.212316,4.145585,4.317378,3.742787,4.398770,4.700417
17127715,12.762858,12.716748,12.679007,12.850743,12.789412,12.891735,12.713205,12.859535,12.704123,12.776425,...,12.750773,12.966153,12.614961,12.805314,12.862282,12.775899,12.837520,12.669491,12.849455,12.764179
17127717,4.358505,5.072454,4.676954,4.664151,4.812907,4.445325,4.958248,4.918783,4.707963,4.981283,...,4.304341,5.009255,5.119118,4.550811,4.826442,4.757878,4.910735,4.438473,4.670488,4.687702


In [21]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'gene_assignment'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [22]:
genetic_data

,GSM3518336,GSM3518337,GSM3518338,GSM3518339,GSM3518340,GSM3518341,GSM3518342,GSM3518343,GSM3518344,GSM3518345,...,GSM3518370,GSM3518371,GSM3518372,GSM3518373,GSM3518374,GSM3518375,GSM3518376,GSM3518377,GSM3518378,GSM3518379
Gene,,,,,,,,,,,,,,,,,,,,,
,3.854410,3.867519,3.905454,3.777970,3.817667,3.651231,3.862177,3.801644,3.818212,3.795680,...,3.796703,3.852327,3.839826,3.833635,3.837002,3.891307,3.836580,3.878469,3.838131,3.822336
,4.238927,4.191213,4.257660,4.141089,4.191537,4.012863,4.217353,4.202952,4.200193,4.185976,...,4.185848,4.189433,4.197750,4.220947,4.208763,4.259131,4.231263,4.231901,4.219814,4.191399
---,2.178849,2.240104,2.184201,2.274834,2.237889,2.309360,2.211402,2.223382,2.219802,2.229682,...,2.243100,2.214198,2.246761,2.220764,2.200025,2.182337,2.200196,2.196976,2.225934,2.213585
---,3.440588,3.427184,3.448184,3.462353,3.463511,3.414615,3.459037,3.437524,3.447143,3.416770,...,3.429741,3.481690,3.460577,3.466210,3.452645,3.451499,3.440189,3.439994,3.482802,3.442220
1,5.578563,5.566120,5.584058,5.543167,5.451951,5.324047,5.523833,5.674940,5.505239,5.674584,...,5.691243,5.664534,5.752765,5.554234,5.689057,5.779966,5.530858,5.626126,5.535039,5.360565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zona occludens 2,4.422342,3.545356,5.122567,4.288449,4.528611,4.400901,4.179276,3.767140,4.017114,3.884510,...,4.995236,4.710068,4.362672,4.223059,4.308590,4.542643,4.272657,4.632529,4.306412,4.437861
zona occludens 3,2.401101,3.291652,2.431860,2.233925,2.299639,2.254953,2.350489,2.273010,2.155829,2.553315,...,2.401214,2.248427,2.696128,2.345314,2.443109,2.257001,2.702208,2.192655,2.325037,2.227378
zygin I,2.935811,2.573002,3.001832,2.905964,2.546398,2.230706,2.678741,2.539368,2.567542,2.414924,...,2.410175,2.960323,2.688676,2.742208,3.110787,3.393251,2.765240,2.535236,2.364203,2.869929


In [23]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

998 input query terms found no hit:	['---', '---', '1', '1', '1 alpha subcomplex', '1 beta subcomplex', '1 like', '1 pseudogene', '1 pse
1000 input query terms found no hit:	['100133941', '100133941', '100133957', '100133957', '100133985', '100133991', '100133991', '1001340
1000 input query terms found no hit:	['100500825', '100500826', '100500827', '100500828', '100500829', '100500830', '100500831', '1005008
1000 input query terms found no hit:	['100507300', '100507300', '100507303', '100507303', '100507305', '100507307', '100507308', '1005073
1000 input query terms found no hit:	['10079', '10079', '1008', '1008', '10081', '10081', '10082', '10082', '100820709', '100820829', '10
1000 input query terms found no hit:	['10520', '10520', '10521', '10521', '10522', '10522', '10523', '10523', '10524', '10524', '10525', 
1000 input query terms found no hit:	['11064', '11065', '11065', '11066', '11067', '11067', '11068', '11068', '11069', '11069', '1107', '
1000 input query terms found no hit

In [24]:
genetic_data

,GSM3518336,GSM3518337,GSM3518338,GSM3518339,GSM3518340,GSM3518341,GSM3518342,GSM3518343,GSM3518344,GSM3518345,...,GSM3518370,GSM3518371,GSM3518372,GSM3518373,GSM3518374,GSM3518375,GSM3518376,GSM3518377,GSM3518378,GSM3518379
CA14,1.986400,2.091525,2.028677,1.834493,1.725591,2.423715,2.229224,2.118898,2.043493,1.876645,...,1.873719,1.662211,1.818252,1.755672,1.768866,1.866015,2.094002,1.774624,2.521507,2.178036
CD79A,1.947850,2.056601,1.843132,2.541186,2.152411,1.940478,1.776484,1.832181,2.263580,2.308381,...,2.715668,1.759831,1.689301,2.341217,2.065179,1.948206,2.125148,2.302319,1.868907,2.260405
EOS,3.990264,4.222771,4.142307,3.648134,4.004018,3.416929,4.155719,4.278521,4.382864,4.437475,...,4.461533,3.970769,4.102581,3.931683,4.511763,3.604085,4.378747,4.116883,4.269606,4.240031
FUSE,6.502828,6.232021,6.650144,6.464514,6.564869,6.018880,6.516231,6.297208,6.548300,6.070564,...,6.489048,6.470714,6.463539,6.641926,6.527462,6.693443,6.498309,6.431285,6.465823,6.381695
FAS,3.567216,3.945198,3.806143,3.448977,3.028453,2.911470,3.314774,2.827372,3.624759,3.073130,...,2.953796,3.420557,3.018158,3.633000,3.510964,3.960850,3.519374,4.031405,3.453952,3.354064
GRID2,2.103987,2.434128,2.386527,3.269374,2.323664,2.542017,2.634600,3.166663,2.792542,2.336443,...,2.182409,2.877743,2.138206,2.871710,2.857754,1.915166,2.401003,2.736337,2.218750,2.747855
PARN,1.523446,1.717991,1.770234,1.830467,1.150353,1.849442,1.674650,1.521761,1.757431,1.776754,...,1.547016,1.705665,1.350861,1.459240,1.534862,1.374706,1.561535,1.598032,1.537481,1.449574
PTPRF,3.077774,3.181721,3.011699,3.066368,3.066777,2.974883,3.003090,3.153287,3.347632,3.205268,...,2.909310,3.111417,3.221858,3.075737,2.982429,3.126628,2.974462,3.011335,3.143614,3.116424
RCC1,3.642806,3.577252,3.697906,3.989598,3.908650,3.456961,4.037533,3.633629,3.687289,3.555085,...,3.844183,3.729384,3.374019,3.630233,3.936260,3.767948,3.712397,3.997099,3.897391,3.651943
TBP,5.065956,5.122564,4.999182,5.042796,5.086878,4.791060,5.008241,5.079071,4.933273,4.983665,...,5.054642,5.039934,4.934077,4.908610,5.076208,4.983788,5.061383,4.976736,5.048106,5.063512


In [25]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [26]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 44 samples.


In [27]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Vitamin D Levels', the least common label is '1.0' with 20 occurrences. This represents 45.45% of the dataset.
The distribution of the feature 'Vitamin D Levels' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 20 occurrences. This represents 45.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [28]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

A new JSON file was created at: ../../output1/Haoyang/Vitamin-D-Levels/cohort_info.json


In [29]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [30]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE123993,True,True,False,False,True,44,


In [31]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

In [32]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [33]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

In [ ]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 4. Discussion and report

In [ ]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)